# CSR Benchmark - Comparing to Bmad

In [ ]:
from pmd_beamphysics import ParticleGroup
from pytao import Tao

import impact.z as IZ
from impact.tests.z.conftest import bmad_files
from impact.z.interfaces.bmad import plot_impactz_and_tao_stats

import matplotlib.pyplot as plt

## Tao

In [ ]:
!cat $bmad_files/csr_bench.bmad

In [ ]:
tao = Tao(lattice_file=bmad_files / "csr_bench.bmad", plot="mpl")

In [ ]:
n_particle = 100_000
a_norm_emit = 1.0e-6
b_norm_emit = 1.0e-6
bunch_charge = 1e-9
sig_pz = 1e-12
sig_z = 0.3e-3  # ~1ps

ds_track_step = 0.01
n_bin = 128

cmds = [
    f"set beam_init n_particle = {n_particle}",
    "set beam_init random_engine = quasi",
    f"set beam_init a_norm_emit = {a_norm_emit}",
    f"set beam_init b_norm_emit = {b_norm_emit}",
    f"set beam_init bunch_charge = {bunch_charge}",
    f"set beam_init sig_pz = {sig_pz}",
    f"set beam_init sig_z = {sig_z}",
    f"set space_charge_com ds_track_step = {ds_track_step}",
    f"set space_charge_com n_bin = {n_bin}",
    f"set ele * ds_step = {ds_track_step}",
]

for cmd in cmds:
    res = tao.cmd(cmd)
    if res:
        print("Tao>", cmd)
        print(res)

In [ ]:
tao.track_beam()

In [ ]:
P0 = ParticleGroup(data=tao.bunch_data("beginning"))

In [ ]:
Ptao = ParticleGroup(data=tao.bunch_data("end"))

## Impact-Z

In [ ]:
input = IZ.ImpactZInput.from_tao(tao)

In [ ]:
I = IZ.ImpactZ(input, initial_particles=P0)
I.input.nz = n_bin

In [ ]:
%%time
output = I.run();

## Initial particles

In [ ]:
P0.plot("t", "energy", bins=100)

## Final particles

### Tao

In [ ]:
Ptao
Ptao.plot("t", "energy")

### IMPACT-Z

In [ ]:
P2 = I.output.particles["final_particles"]
P2.plot("t", "energy")

# Compare

In [ ]:
def compare(
    xkey="x",
    ykey="y",
    skip=1,
):
    fig, ax = plt.subplots()

    plist = (
        (Ptao, "Bmad", "X", "blue"),
        (P2, "Impact-Z", ".", "orange"),
    )

    for p, label, marker, color in plist:
        ax.scatter(
            p[xkey][::skip],
            p[ykey][::skip],
            label=label,
            marker=marker,
            alpha=0.5,
            color=color,
        )

    for p, label, marker, color in plist:
        ax.scatter(
            p["mean_" + xkey],
            p["mean_" + ykey],
            marker="+",
            color=color,
            facecolor="black",
        )

    plt.legend()

    ax.set_xlabel(xkey)
    ax.set_ylabel(ykey)


compare(skip=100)

In [ ]:
compare("t", "energy", skip=100)

In [ ]:
plot_impactz_and_tao_stats(I, tao)